### 2024-04-21 すべてのドローイングのtsvファイルをローカルに保存する（解析時間短縮のため）

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import io

In [18]:
# ドローイング情報を取得する
def get_drawing_data(url, drawing_id, rotate_type):
    class DrawingData():
        def __init__(self, url, drawing_id, rotate_type):
            self.columns = [ 'time', 'alpha', 'max_points', 'ptx_anoto', 'pty_anoto', 'shape_str',
                        'pressure_style', 'len_mm_acc', 'len_anoto', 'pressure_avg', 'shape_int']
            self.url = url
            self.drawing_id = drawing_id
            self.rotate_type = rotate_type
            self.df_org = None
            self.df_mm = None
            
            
            # urlからデータを生成
            self.get_df(self.url)
            self.conv_str_to_coord()

            ################################
            
        # URLからデータフレームを取得 & ストローク長の列を追加
        def get_df(self, url):
            content = requests.get(url, auth=('19t2003a', 'ireneRED77')).content
            df = pd.read_table(io.StringIO(content.decode('utf-8')), header = None)
            df.columns = self.columns
            st_len_mm = [l*0.3 for l in df['len_anoto']]
            df['len_mm'] = st_len_mm
            # ストローク固有のidを付与
            df_reindex = df.reset_index()
            df_reindex = df_reindex.rename(columns={'index': 'stroke_id'})
            
            # ドローイングidの列を追加
            df_reindex['drawing_id'] = [self.drawing_id for i in range(len(df_reindex))]
            
            self.df_org = df_reindex.copy()
            
        # 文字列座標データをリストに変換 & anotoから普通座標に変換
        def conv_str_to_coord(self):
            df = self.df_org.copy()
            x_list = []
            y_list = []
            for str_x, str_y in zip(df['ptx_anoto'], df['pty_anoto']):
                x_list.append([float(x)*0.3 for x in str_x.split(',')])
                y_list.append([float(y)*0.3 for y in str_y.split(',')])
            df['ptx_mm_org'] = x_list
            df['pty_mm_org'] = y_list
            self.df_mm = df
            
            
    data = DrawingData(url, drawing_id, rotate_type)
    
    return data.df_mm

In [14]:
asai_target_info = pd.read_excel('../data/all_drawing_info.xlsx', index_col=0).query('is_asai == 1')

In [45]:
for url, drawing_id, rotate_type in zip(
    asai_target_info['url'], asai_target_info['drawing_id'], asai_target_info['rotate_type']):
    df = get_drawing_data(url, drawing_id, rotate_type)
    df.to_csv(f'../data/all_tsv/id_{drawing_id}.tsv', sep='\t')
    print(drawing_id)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [4]:
import pandas as pd
data = pd.read_csv('../data/all_tsv/id_0.tsv', delimiter='\t', index_col=0)

In [6]:
data.query('shape_int == 3')

,stroke_id,time,alpha,max_points,ptx_anoto,pty_anoto,shape_str,pressure_style,len_mm_acc,len_anoto,pressure_avg,shape_int,len_mm,drawing_id,ptx_mm_org,pty_mm_org
17,17,166,104.422043,18,"1227.875,1228.0,1228.0,1225.875,1223.25,1212.0...","255.375,255.375,255.5,255.875,255.875,256.0,25...",Complex,61.489418,106.043053,54.862957,50.777778,3,16.458887,0,"[368.3625, 368.4, 368.4, 367.7625, 366.9749999...","[76.6125, 76.6125, 76.64999999999999, 76.7625,..."
93,93,329,217.983871,16,"386.875,386.875,387.0,387.125,388.125,390.25,3...","112.125,112.375,112.5,112.625,110.25,106.125,1...",Complex,96.164857,1461.122868,25.120381,106.000000,3,7.536114,0,"[116.0625, 116.0625, 116.1, 116.13749999999999...","[33.637499999999996, 33.7125, 33.75, 33.7875, ..."
94,94,330,224.235484,25,"391.625,392.125,392.125,392.125,391.625,387.62...","104.625,104.5,104.625,104.75,105.75,108.625,10...",Complex,96.000760,1478.627389,58.348402,109.040000,3,17.504520,0,"[117.4875, 117.63749999999999, 117.63749999999...","[31.3875, 31.349999999999998, 31.3875, 31.4249..."
101,101,336,159.829332,43,"421.125,401.375,365.0,352.875,347.375,347.75,3...","99.25,98.75,98.25,98.5,99.5,101.375,103.625,10...",Complex,73.881205,2029.645906,534.982408,77.720930,3,160.494723,0,"[126.33749999999999, 120.4125, 109.5, 105.8625...","[29.775, 29.625, 29.474999999999998, 29.549999..."
102,102,337,124.494417,26,"365.5,359.625,362.125,397.125,437.375,481.5,51...","99.25,99.25,99.5,99.25,98.625,97.25,94.5,91.75...",Complex,61.086895,2203.713822,580.226386,60.538462,3,174.067916,0,"[109.64999999999999, 107.8875, 108.6375, 119.1...","[29.775, 29.775, 29.849999999999998, 29.775, 2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1478,1478,1128,183.302092,74,"263.75,271.25,273.375,272.5,261.625,252.75,243...","217.75,212.5,211.375,212.875,223.875,231.0,237...",Complex,82.995233,62845.783048,969.978902,89.135135,3,290.993671,0,"[79.125, 81.375, 82.0125, 81.75, 78.4875, 75.8...","[65.325, 63.75, 63.412499999999994, 63.8625, 6..."
1484,1484,1131,197.485692,62,"230.875,240.25,241.875,237.25,226.875,193.875,...","266.625,263.875,262.25,261.0,260.125,258.875,2...",Complex,83.707872,63373.661975,1170.176478,96.032258,3,351.052943,0,"[69.2625, 72.075, 72.5625, 71.175, 68.0625, 58...","[79.9875, 79.1625, 78.675, 78.3, 78.0375, 77.6..."
1492,1492,1135,229.441244,28,"260.25,257.125,256.0,256.875,262.625,288.25,31...","219.125,220.375,220.625,218.5,213.5,194.125,17...",Complex,95.851620,63860.581451,427.219048,111.571429,3,128.165714,0,"[78.075, 77.1375, 76.8, 77.0625, 78.7875, 86.4...","[65.7375, 66.1125, 66.1875, 65.55, 64.05, 58.2..."
1504,1504,1143,126.201188,19,"205.625,215.375,215.5,192.625,148.0,93.125,95....","286.5,286.5,286.75,288.25,290.5,291.0,291.625,...",Complex,64.823737,64921.399343,393.994705,61.368421,3,118.198411,0,"[61.6875, 64.6125, 64.64999999999999, 57.78749...","[85.95, 85.95, 86.02499999999999, 86.475, 87.1..."
